<a href="https://colab.research.google.com/github/chanseongparkk/BigDataAnalyst_practise/blob/main/%EC%9D%B4%EC%A7%81%EC%97%AC%EB%B6%80_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
### 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")

### 데이터에서 id 분리 후 따로 저장하고 데이터셋에서 id 삭제

x_train_id = x_train.enrollee_id
x_train = x_train.drop(["enrollee_id"],axis = 1)
x_test_id = x_test.enrollee_id
x_test = x_test.drop(["enrollee_id"],axis = 1)
y_train_id = y_train.enrollee_id
y_train = y_train.drop(["enrollee_id"],axis = 1)

### 결측치가 15% 이상 넘어가는 변수 삭제
# print(x_train.isna().sum(),x_test.isna().sum(),x_train.shape,x_test.shape)

x_train = x_train.drop(["gender","company_size","company_type","major_discipline"],axis = 1)
x_test = x_test.drop(["gender","company_size","company_type","major_discipline"],axis = 1)

#print(x_train.isna().sum(),x_test.isna().sum(),x_train.shape,x_test.shape)

### unique 25개 넘어가는 변수 삭제

#print(x_train.describe(include = "object"))

x_train = x_train.drop(["city"],axis =1)
x_test = x_test.drop(["city"],axis = 1)

# print(x_train.describe(include = "object"),"\n",x_test.describe(include = "object"))

### 결측치 대체

#print(x_train.isna().sum(),x_test.isna().sum(),x_train.info()) #object 변수에만 결측치가 존재한다.

# 최빈값으로 대체해준다.
import numpy as np
x_train.iloc[:,2] = x_train.iloc[:,2].replace(np.nan, x_train.iloc[:,2].value_counts().index[0])
x_train.iloc[:,3] = x_train.iloc[:,3].replace(np.nan, x_train.iloc[:,3].value_counts().index[0])
x_train.iloc[:,4] = x_train.iloc[:,4].replace(np.nan, x_train.iloc[:,4].value_counts().index[0])
x_train.iloc[:,5] = x_train.iloc[:,5].replace(np.nan, x_train.iloc[:,5].value_counts().index[0])
x_test.iloc[:,2] = x_test.iloc[:,2].replace(np.nan, x_test.iloc[:,2].value_counts().index[0])
x_test.iloc[:,3] = x_test.iloc[:,3].replace(np.nan, x_test.iloc[:,3].value_counts().index[0])
x_test.iloc[:,4] = x_test.iloc[:,4].replace(np.nan, x_test.iloc[:,4].value_counts().index[0])
x_test.iloc[:,5] = x_test.iloc[:,5].replace(np.nan, x_test.iloc[:,5].value_counts().index[0])

# print(x_train.isna().sum(),x_test.isna().sum()) #결측치 X

### 변수 변환

#print(x_train.iloc[:,[0,6]].skew(),"\n\n",x_train.iloc[:,[0,6]].kurt())
#print(np.log1p(x_train.iloc[:,[0,6]]).skew(),"\n\n",np.log1p(x_train.iloc[:,[0,6]]).kurt()) # training_hour 변수를 로그변환 해준다.

x_train.iloc[:,6] = np.log1p(x_train.iloc[:,6])
x_test.iloc[:,6] = np.log1p(x_test.iloc[:,6])

### 변수 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train.iloc[:,[0,6]] = scaler.fit_transform(x_train.iloc[:,[0,6]]) 
x_test.iloc[:,[0,6]] = scaler.transform(x_test.iloc[:,[0,6]])

# print(x_train.describe(), "\n\n", x_test.describe())

### 더미변수

x_train = pd.get_dummies(x_train, drop_first = True)
x_test = pd.get_dummies(x_test,drop_first = True)

# 문자 들어간 변수명 변경
x_train = x_train.rename(columns = {"experience_<1":"experience_under_1","experience_>20":"experience_upper_20","last_new_job_>4":"last_new_job_upper_4"}) 
x_test = x_test.rename(columns = {"experience_<1":"experience_under_1","experience_>20":"experience_upper_20","last_new_job_>4":"last_new_job_upper_4"}) 

# print(x_train.columns,x_test.columns)
# print(set(x_train.columns) - set(x_test.columns)) # x_train과 x_test의 범주형 변수 개체 차이 없음

# print(x_train.shape, x_test.shape) #동일한 설명변수 갯수 35개

### 모델링(RandomForestClassifier)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
rf = RandomForestClassifier(n_estimators = 100, max_depth = 5, min_samples_split = 5, random_state = 999)
#rf = RandomForestClassifier(random_state = 999)
rf.fit(x_train,y_train)

### 모델링(XGBClassifier)
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators = 100, max_depth = 5, subsample = 0.3, random_state = 999)
#xgb = XGBClassifier( random_state = 999)
xgb.fit(x_train, y_train,eval_metric = "auc")
print("rf accuracy: ",rf.score(x_train,y_train),"\n", "rf roc_auc_score: ", roc_auc_score(y_train,rf.predict(x_train)),"\n",
      "xgb accuracy: ",xgb.score(x_train,y_train),"\n", "xgb roc_auc_score: ", roc_auc_score(y_train,xgb.predict(x_train)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


rf accuracy:  0.7535335689045937 
 rf roc_auc_score:  0.508378001363428 
 xgb accuracy:  0.8007548988114359 
 xgb roc_auc_score:  0.6808136016154399


In [ ]:
#모델 평가
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv')
print("rf roc_auc_score : ", roc_auc_score(y_test['target'], rf.predict(x_test)),"\n",
      "xgb roc_auc_score : ", roc_auc_score(y_test['target'], xgb.predict(x_test)))

rf roc_auc_score :  0.5061869954168117 
 xgb roc_auc_score :  0.64662103913484
